In [1]:
import os
import shutil

# = = = = = = = = = = = = = = =

if os.path.exists('./file/5.collate'):
    shutil.rmtree('./file/5.collate')

os.makedirs('./file/5.collate')

# 1. Json

In [2]:
import pandas as pd

from tqdm import tqdm
import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/3.part/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    
    # = = = = = = = = = = = = = = =

    list_column, list_column_quantity, list_column_specific = input_.columns.tolist(), [], []

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
        df_temp, dict_quantity, dict_dict = pd.DataFrame([input_.iloc[i]]).reset_index(drop=True), json.loads(input_.loc[i, 'Json_Quantity']), json.loads(input_.loc[i, 'Json_Specific'])

        # = = = = = = = = = = = = = = =

        for key, value in dict_quantity.items():
            key = f'Quantity-{int(key) + 1}'

            if key not in list_column_quantity:
                list_column_quantity.append(key)
                
            df_temp.loc[0, key] = value

        # = = = = = = = = = = = = = = =
        
        for dict_ in dict_dict.values():
            for key, value in dict_.items():
                if key in list_column:
                    key = f'''{key} (Specific)'''
                
                if key not in list_column_specific:
                    list_column_specific.append(key)
                    
                df_temp.loc[0, key] = value
    
        list_df.append(df_temp)
    
    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output[list_column[:7] + list_column_quantity + sorted(list_column_specific) + list_column[7:]].to_excel(f'./file/5.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 172.48it/s]



crawler_2.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 144.39it/s]


Done ~


# 2. Compatible

In [3]:
import pandas as pd

from tqdm import tqdm

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/4.vehicle/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    input_['Page'] = input_['Page'].astype(int)
    input_['Row'] = input_['Row'].astype(int)

    # = = = = = = = = = = = = = = =

    input_2 = input_.drop_duplicates(['Json_Compatibility'],
                                     keep='first',
                                     ignore_index=True)
    input_2 = input_2.sort_values(by=['No'],
                                  ascending=[True],
                                  ignore_index=True)

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
        df_temp, country = input_[input_['Json_Compatibility'] == input_2.loc[i, 'Json_Compatibility']].reset_index(drop=True), json.loads(input_2.loc[i, 'Json_Compatibility'])['data']['scopedContext']['catalogDetails']['marketplaceId']

        dict_vehicle = {}
        if country == 'EBAY-DE':
            for j in range(len(df_temp)):
                make, model, engine, year, year_2 = df_temp.loc[j, 'Marke'], df_temp.loc[j, 'Modell'], df_temp.loc[j, 'Motor'], int(df_temp.loc[j, 'Baujahr'].split('-')[0].split('/')[0]), int(df_temp.loc[j, 'Baujahr'].split('-')[1].split('/')[0])

                if make not in dict_vehicle:
                    dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
                elif model not in dict_vehicle[make]:
                    dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
                elif engine not in dict_vehicle[make][model]:
                    dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
                else:
                    if year not in dict_vehicle[make][model][engine]:
                        dict_vehicle[make][model][engine].append(year)
                    if year_2 not in dict_vehicle[make][model][engine]:
                        dict_vehicle[make][model][engine].append(year_2)
        else:
            for j in range(len(df_temp)):
                make, model, engine, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Engine'], int(df_temp.loc[j, 'Year'])

                if make not in dict_vehicle:
                    dict_vehicle[make] = {model: {engine: [year]}}
                elif model not in dict_vehicle[make]:
                    dict_vehicle[make][model] = {engine: [year]}
                elif engine not in dict_vehicle[make][model]:
                    dict_vehicle[make][model][engine] = [year]
                elif year not in dict_vehicle[make][model][engine]:
                    dict_vehicle[make][model][engine].append(year)
    
        # = = = = = = = = = = = = = = =

        list_vehicle, list_vehicle_2 = [], []
        for make in dict_vehicle:
            for model in dict_vehicle[make]:
                list_year = []
                for engine in dict_vehicle[make][model]:
                    list_year = list_year + dict_vehicle[make][model][engine]
                    
                    year_begin = min(dict_vehicle[make][model][engine])
                    year_end = max(dict_vehicle[make][model][engine])
                    list_vehicle_2.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())
                
                year_begin = min(list_year)
                year_end = max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        # = = = = = = = = = = = = = = =

        df_temp = pd.DataFrame([{'No': input_2.loc[i, 'No'],
                                 'Item Number': input_2.loc[i, 'Item Number'],
                                 'Json_Compatibility': input_2.loc[i, 'Json_Compatibility'],
                                 'Vehicle W/o Engine': '\n'.join(sorted(list_vehicle)),
                                 'Vehicle W/ Engine': '\n'.join(sorted(list_vehicle_2))}])
        list_df.append(df_temp)

    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output.to_excel(f'./file/5.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 362.68it/s]


crawler_2-vehicle.xlsx



Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 152.55it/s]


Done ~
